# DeepFabric ile Sentetik Data Üretimi

Bu notebook, DeepFabric kütüphanesini kullanarak lokal LLM API ile sentetik veri üretimini göstermektedir.

## İçindekiler
1. Kurulum
2. Temel Kullanım
3. Lokal LLM ile Bağlantı (Ollama)
4. Farklı Türde Sentetik Data Üretimi
5. Üretilen Verinin Görselleştirilmesi ve Analizi
6. Fine-tuning için Hazırlık

## 1. Kurulum

### DeepFabric Kütüphanesini Kurma

In [ ]:
# DeepFabric kütüphanesini GitHub'dan kuruyoruz
!pip install git+https://github.com/lukehinds/deepfabric.git -q

In [ ]:
# Gerekli yardımcı kütüphaneleri kuruyoruz
!pip install datasets pandas matplotlib seaborn pyyaml -q

### Gerekli Kütüphaneleri İçe Aktarma

In [ ]:
import json
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datasets import load_dataset
import subprocess
import os

# Görselleştirme ayarları
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Kütüphaneler başarıyla yüklendi!")

## 2. Lokal LLM Kurulumu (Ollama)

DeepFabric, lokal LLM kullanımı için Ollama'yı destekliyor. Önce Ollama'nın kurulu olduğundan emin olun.

### Ollama Kurulum Talimatları:
```bash
# Linux için:
curl -fsSL https://ollama.com/install.sh | sh

# MacOS için:
brew install ollama

# Ollama'yı başlatın:
ollama serve

# Bir model indirin (örnek: Llama 3.1):
ollama pull llama3.1:8b
# veya daha hafif bir model:
ollama pull qwen2.5:3b
```

In [ ]:
# Ollama'nın çalışıp çalışmadığını kontrol et
try:
    result = subprocess.run(['ollama', 'list'], capture_output=True, text=True)
    print("Mevcut Ollama Modelleri:")
    print(result.stdout)
except FileNotFoundError:
    print("⚠️ Ollama bulunamadı! Lütfen yukarıdaki talimatları takip ederek Ollama'yı kurun.")

## 3. Yapılandırma Dosyası Oluşturma

DeepFabric'i YAML yapılandırma dosyası ile kullanacağız. Bu, daha fazla kontrol sağlar.

In [ ]:
# Yapılandırma dizini oluştur
config_dir = Path('deepfabric_configs')
config_dir.mkdir(exist_ok=True)

# Çıktı dizini oluştur
output_dir = Path('synthetic_datasets')
output_dir.mkdir(exist_ok=True)

print("✓ Dizinler oluşturuldu!")

### Örnek 1: Basit Chain of Thought (CoT) Yapılandırması

In [ ]:
# Basit CoT yapılandırması
cot_config = {
    'topic_tree': {
        'provider': 'ollama',
        'model': 'llama3.1:8b',  # Modelinizi buraya yazın
        'temperature': 0.7,
        'degree': 3,  # Her düğümden 3 alt konu üretilecek
        'depth': 2,   # 2 seviye derinlik
    },
    'data_engine': {
        'provider': 'ollama',
        'model': 'llama3.1:8b',
        'generation_system_prompt': 'Sen yardımcı bir Python programlama eğitmenisin. Öğrencilere adım adım düşünmeyi öğretiyorsun.',
        'temperature': 0.3,
        'num_steps': 20,  # 20 örnek üret
        'batch_size': 5,
        'max_retries': 3,
        'reasoning_template': 'chain_of_thought',
    },
    'output': {
        'formatter': 'builtin://trl_sft_tools',
        'sys_msg': True,
    }
}

# YAML dosyası olarak kaydet
cot_config_path = config_dir / 'cot_config.yaml'
with open(cot_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(cot_config, f, allow_unicode=True)

print(f"✓ CoT yapılandırması kaydedildi: {cot_config_path}")
print("\nYapılandırma içeriği:")
print(yaml.dump(cot_config, allow_unicode=True))

## 4. Sentetik Data Üretimi

### Örnek 1: Python Programlama için CoT Verisi

In [ ]:
# Chain of Thought verisi üret
output_file_cot = output_dir / 'python_cot_dataset.jsonl'

cmd = [
    'deepfabric', 'generate',
    str(cot_config_path),
    '--topic-prompt', 'Python Programlama Temelleri: Veri yapıları, algoritmalar ve fonksiyonlar',
    '--output', str(output_file_cot)
]

print("Sentetik veri üretiliyor... (Bu birkaç dakika sürebilir)")
print(f"Komut: {' '.join(cmd)}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Hatalar/Uyarılar:")
    print(result.stderr)

print("\n" + "="*80)
print(f"✓ Veri üretimi tamamlandı! Dosya: {output_file_cot}")

### Örnek 2: Tool Calling (Fonksiyon Çağırma) Yapılandırması

In [ ]:
# Tool calling yapılandırması
tool_config = {
    'topic_tree': {
        'provider': 'ollama',
        'model': 'llama3.1:8b',
        'temperature': 0.8,
        'degree': 4,
        'depth': 2,
    },
    'data_engine': {
        'provider': 'ollama',
        'model': 'llama3.1:8b',
        'generation_system_prompt': 'Sen bir API yardımcısısın. Kullanıcılara doğru fonksiyonları seçip çağırmayı öğretiyorsun.',
        'temperature': 0.2,
        'num_steps': 15,
        'batch_size': 5,
        'reasoning_template': 'tool_calling',
    },
    'output': {
        'formatter': 'builtin://trl_sft_tools',
        'sys_msg': True,
    }
}

tool_config_path = config_dir / 'tool_config.yaml'
with open(tool_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(tool_config, f, allow_unicode=True)

print(f"✓ Tool calling yapılandırması kaydedildi: {tool_config_path}")

In [ ]:
# Tool calling verisi üret
output_file_tool = output_dir / 'tool_calling_dataset.jsonl'

cmd = [
    'deepfabric', 'generate',
    str(tool_config_path),
    '--topic-prompt', 'API ve fonksiyon çağırma: Veritabanı sorguları, dosya işlemleri ve ağ istekleri',
    '--output', str(output_file_tool)
]

print("Tool calling verisi üretiliyor...")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Hatalar/Uyarılar:")
    print(result.stderr)

print("\n" + "="*80)
print(f"✓ Tool calling verisi oluşturuldu: {output_file_tool}")

### Örnek 3: Multi-Turn Conversation (Çok Turlu Konuşma)

In [ ]:
# Multi-turn conversation yapılandırması
multiturn_config = {
    'topic_tree': {
        'provider': 'ollama',
        'model': 'llama3.1:8b',
        'temperature': 0.7,
        'degree': 3,
        'depth': 3,
    },
    'data_engine': {
        'provider': 'ollama',
        'model': 'llama3.1:8b',
        'generation_system_prompt': 'Sen teknoloji danışmanısın. Kullanıcılarla detaylı, çok turlu konuşmalar yaparak sorunlarını çözüyorsun.',
        'temperature': 0.5,
        'num_steps': 10,
        'batch_size': 3,
        'reasoning_template': 'multi_turn',
    },
    'output': {
        'formatter': 'builtin://chatml',
        'sys_msg': True,
    }
}

multiturn_config_path = config_dir / 'multiturn_config.yaml'
with open(multiturn_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(multiturn_config, f, allow_unicode=True)

print(f"✓ Multi-turn yapılandırması kaydedildi: {multiturn_config_path}")

In [ ]:
# Multi-turn verisi üret
output_file_multiturn = output_dir / 'multiturn_dataset.jsonl'

cmd = [
    'deepfabric', 'generate',
    str(multiturn_config_path),
    '--topic-prompt', 'Yazılım geliştirme sorunları: Debugging, optimizasyon ve mimari kararlar',
    '--output', str(output_file_multiturn)
]

print("Multi-turn conversation verisi üretiliyor...")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Hatalar/Uyarılar:")
    print(result.stderr)

print("\n" + "="*80)
print(f"✓ Multi-turn verisi oluşturuldu: {output_file_multiturn}")

## 5. Üretilen Verinin Analizi ve Görselleştirilmesi

In [ ]:
def load_jsonl(file_path):
    """JSONL dosyasını yükle"""
    data = []
    if not Path(file_path).exists():
        print(f"⚠️ Dosya bulunamadı: {file_path}")
        return data
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def analyze_dataset(data, dataset_name):
    """Veri setini analiz et"""
    print(f"\n{'='*80}")
    print(f"Dataset: {dataset_name}")
    print(f"{'='*80}")
    print(f"Toplam örnek sayısı: {len(data)}")
    
    if len(data) > 0:
        # İlk örneği göster
        print("\n📝 İlk Örnek:")
        print(json.dumps(data[0], indent=2, ensure_ascii=False)[:1000] + "...")
        
        # Mesaj uzunluklarını hesapla
        message_lengths = []
        for item in data:
            if 'messages' in item:
                for msg in item['messages']:
                    if 'content' in msg:
                        message_lengths.append(len(msg['content']))
        
        if message_lengths:
            print(f"\n📊 İstatistikler:")
            print(f"  - Ortalama mesaj uzunluğu: {sum(message_lengths)/len(message_lengths):.0f} karakter")
            print(f"  - En kısa mesaj: {min(message_lengths)} karakter")
            print(f"  - En uzun mesaj: {max(message_lengths)} karakter")
            print(f"  - Toplam mesaj sayısı: {len(message_lengths)}")
    
    return data

# Tüm veri setlerini yükle ve analiz et
datasets = {
    'Chain of Thought': output_file_cot,
    'Tool Calling': output_file_tool,
    'Multi-Turn': output_file_multiturn
}

loaded_datasets = {}
for name, path in datasets.items():
    data = load_jsonl(path)
    loaded_datasets[name] = analyze_dataset(data, name)

In [ ]:
# Veri setlerini görselleştir
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Veri seti boyutları
dataset_sizes = {name: len(data) for name, data in loaded_datasets.items() if len(data) > 0}
if dataset_sizes:
    axes[0].bar(dataset_sizes.keys(), dataset_sizes.values(), color=['#3498db', '#e74c3c', '#2ecc71'])
    axes[0].set_xlabel('Veri Seti', fontsize=12)
    axes[0].set_ylabel('Örnek Sayısı', fontsize=12)
    axes[0].set_title('Veri Seti Boyutları', fontsize=14, fontweight='bold')
    axes[0].tick_params(axis='x', rotation=45)

# Mesaj uzunluk dağılımı
all_lengths = []
labels = []
for name, data in loaded_datasets.items():
    if len(data) > 0:
        lengths = []
        for item in data:
            if 'messages' in item:
                for msg in item['messages']:
                    if 'content' in msg:
                        lengths.append(len(msg['content']))
        if lengths:
            all_lengths.append(lengths)
            labels.append(name)

if all_lengths:
    axes[1].boxplot(all_lengths, labels=labels)
    axes[1].set_xlabel('Veri Seti', fontsize=12)
    axes[1].set_ylabel('Mesaj Uzunluğu (karakter)', fontsize=12)
    axes[1].set_title('Mesaj Uzunluk Dağılımı', fontsize=14, fontweight='bold')
    axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(output_dir / 'dataset_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✓ Görselleştirme kaydedildi: {output_dir / 'dataset_analysis.png'}")

## 6. Örnek Mesajları İnceleme

In [ ]:
def display_example(data, dataset_name, index=0):
    """Belirli bir örneği güzel formatta göster"""
    if len(data) == 0:
        print(f"⚠️ {dataset_name} veri seti boş!")
        return
    
    if index >= len(data):
        index = 0
    
    print(f"\n{'='*80}")
    print(f"Dataset: {dataset_name} - Örnek #{index + 1}")
    print(f"{'='*80}\n")
    
    item = data[index]
    
    if 'messages' in item:
        for i, msg in enumerate(item['messages']):
            role = msg.get('role', 'unknown').upper()
            content = msg.get('content', '')
            
            print(f"[{role}]")
            print(f"{content}")
            print(f"\n{'-'*80}\n")
    else:
        print(json.dumps(item, indent=2, ensure_ascii=False))

# Her veri setinden birer örnek göster
for name, data in loaded_datasets.items():
    if len(data) > 0:
        display_example(data, name, index=0)

## 7. Fine-tuning için Veri Hazırlama

Üretilen veriler direkt olarak HuggingFace TRL SFTTrainer ile kullanılabilir.

In [ ]:
# Veri setlerini birleştir (opsiyonel)
combined_data = []
for name, data in loaded_datasets.items():
    combined_data.extend(data)

# Birleştirilmiş veri setini kaydet
combined_output = output_dir / 'combined_dataset.jsonl'
with open(combined_output, 'w', encoding='utf-8') as f:
    for item in combined_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✓ Birleştirilmiş veri seti kaydedildi: {combined_output}")
print(f"  Toplam örnek sayısı: {len(combined_data)}")

In [ ]:
# HuggingFace datasets kütüphanesi ile yükle
try:
    dataset = load_dataset('json', data_files=str(combined_output))
    print("✓ Veri seti başarıyla yüklendi!")
    print(f"\nVeri seti bilgileri:")
    print(dataset)
    
    # Örnek bir satır göster
    if len(dataset['train']) > 0:
        print("\n📝 Örnek bir satır:")
        print(dataset['train'][0])
except Exception as e:
    print(f"⚠️ Hata: {e}")

## 8. Fine-tuning Örnek Kodu (TRL SFTTrainer)

Bu kod üretilen veri ile model fine-tuning yapmak için kullanılabilir.

In [ ]:
# Fine-tuning için örnek kod (çalıştırmak için gerekli kütüphaneleri kurun)
example_training_code = '''
# Gerekli kütüphaneler:
# pip install transformers trl peft accelerate

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from trl import SFTTrainer
from datasets import load_dataset
from peft import LoraConfig

# Model ve tokenizer yükle
model_name = "meta-llama/Llama-3.2-1B-Instruct"  # veya başka bir model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Veri setini yükle
dataset = load_dataset("json", data_files="synthetic_datasets/combined_dataset.jsonl")

# LoRA yapılandırması
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Training parametreleri
training_args = TrainingArguments(
    output_dir="./finetuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    warmup_steps=50,
)

# Trainer oluştur
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_seq_length=2048,
)

# Eğitimi başlat
trainer.train()

# Modeli kaydet
trainer.save_model("./finetuned_model")
'''

print("Fine-tuning Örnek Kodu:")
print("="*80)
print(example_training_code)

# Kodu dosyaya kaydet
with open(output_dir / 'example_finetuning.py', 'w', encoding='utf-8') as f:
    f.write(example_training_code)

print(f"\n✓ Örnek kod kaydedildi: {output_dir / 'example_finetuning.py'}")

## 9. Özet ve Sonraki Adımlar

### Yapılanlar:
1. ✓ DeepFabric kütüphanesi kuruldu
2. ✓ Lokal LLM (Ollama) ile bağlantı yapıldı
3. ✓ 3 farklı türde sentetik veri üretildi:
   - Chain of Thought (CoT)
   - Tool Calling
   - Multi-Turn Conversations
4. ✓ Veri setleri analiz edildi ve görselleştirildi
5. ✓ Fine-tuning için hazır hale getirildi

### Sonraki Adımlar:

1. **Daha Fazla Veri Üretimi:**
   - `num_steps` parametresini artırarak daha fazla örnek üretin
   - Farklı topic_prompt'lar kullanarak çeşitli konularda veri üretin
   - `degree` ve `depth` parametreleri ile daha karmaşık konu ağaçları oluşturun

2. **Model Fine-tuning:**
   - Üretilen veri ile küçük bir model (Llama-3.2-1B) fine-tune edin
   - LoRA veya QLoRA kullanarak verimli eğitim yapın
   - Farklı hyperparameter'lar deneyin

3. **Veri Kalitesi İyileştirme:**
   - Temperature değerlerini ayarlayarak çeşitliliği kontrol edin
   - System prompt'ları özelleştirerek istediğiniz formata uygun veri üretin
   - Farklı reasoning template'leri deneyin

4. **Değerlendirme:**
   - Fine-tune edilmiş modeli test edin
   - Benchmark'larda performansını ölçün
   - İyileştirme için geri bildirim toplayın

### Faydalı Kaynaklar:
- DeepFabric GitHub: https://github.com/lukehinds/deepfabric
- Ollama: https://ollama.com/
- HuggingFace TRL: https://huggingface.co/docs/trl/
- PEFT/LoRA: https://huggingface.co/docs/peft/

## 10. Temizlik ve Bakım

In [ ]:
# Tüm üretilen dosyaları listele
print("Üretilen Dosyalar:")
print("="*80)

for file in sorted(output_dir.glob('*')):
    size = file.stat().st_size
    size_mb = size / (1024 * 1024)
    print(f"  📄 {file.name:40s} - {size_mb:8.2f} MB")

print("\n" + "="*80)
total_size = sum(f.stat().st_size for f in output_dir.glob('*'))
print(f"Toplam boyut: {total_size / (1024 * 1024):.2f} MB")